---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

# read in the data
spam_data = pd.read_csv('spam.csv')

# convert target to numeric
spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split

# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    df_true = spam_data[spam_data["target"]==1]
    ans = float(len(df_true))/ len(spam_data) * 100
    
    return ans

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    
    vect = CountVectorizer().fit(X_train)
    vocab = vect.get_feature_names()
#     len_list = (len(x) for x in vocab)
    longest = max(vocab, key=len)
    
    return longest

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    clsf = MultinomialNB(alpha=0.1)
    clsf.fit(X_train_vectorized, y_train)
    predictions = clsf.predict(vect.transform(X_test))
    
    auc = roc_auc_score(y_test, predictions)
    return auc

In [8]:
answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    tfidf_vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = tfidf_vect.transform(X_train)
    # get feature names as array
    feature_names = np.array(tfidf_vect.get_feature_names())
    # get max tfidf for each word
    tfidf_values = X_train_vectorized.max(0).toarray()[0]
    #tfidf_values = tfidf_vect.idf_
    sorted_tfidf_index = tfidf_values.argsort()
    # smallest index
    smallest_index = sorted_tfidf_index[:20]
    # largest index
    largest_index = sorted_tfidf_index[:-21:-1]
    # smallest series
    smallest_tfidfs = pd.Series(data=tfidf_values[smallest_index],
                               index=feature_names[smallest_index])
    # largest series
    largest_tfidfs = pd.Series(data=tfidf_values[largest_index],
                               index=feature_names[largest_index])

    # now sort them carefully
    small = smallest_tfidfs.iloc[np.lexsort([smallest_tfidfs.index, smallest_tfidfs.values])]
    large = largest_tfidfs.iloc[np.lexsort([largest_tfidfs.index, largest_tfidfs.values])][::-1]    
    
    return (small, large)

In [10]:
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, yup          1.000000
 where        1.000000
 too          1.000000
 thanx        1.000000
 thank        1.000000
 okie         1.000000
 ok           1.000000
 nite         1.000000
 lei          1.000000
 home         1.000000
 havent       1.000000
 er           1.000000
 done         1.000000
 beerage      1.000000
 anytime      1.000000
 anything     1.000000
 645          1.000000
 146tf150p    1.000000
 tick         0.980166
 blank        0.932702
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_five():
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)    
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)
    predictions = model.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, predictions)

In [12]:
answer_five()

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [13]:
def answer_six():
    spam_dt = spam_data[spam_data['target']==1]
    not_spam_dt = spam_data[spam_data['target']==0]
    spam_len_avg = np.mean([len(x) for x in spam_dt["text"].values])
    not_spam_len_avg = np.mean([len(x) for x in not_spam_dt["text"].values])
    
    return (not_spam_len_avg, spam_len_avg)

In [14]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [18]:
from sklearn.svm import SVC

def answer_seven():
    vectorizer = TfidfVectorizer(min_df=5)

    X_train_transformed = vectorizer.fit_transform(X_train)
    X_train_transformed_with_length = add_feature(X_train_transformed, X_train.str.len())

    X_test_transformed = vectorizer.transform(X_test)
    X_test_transformed_with_length = add_feature(X_test_transformed, X_test.str.len())

    clf = SVC(C=10000)

    clf.fit(X_train_transformed_with_length, y_train)

    y_predicted = clf.predict(X_test_transformed_with_length)    
        
    return roc_auc_score(y_test, y_predicted)

In [19]:
answer_seven()

0.9581366823421557

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [30]:
def answer_eight():

    spam_dt = spam_data[spam_data['target']==1]
    not_spam_dt = spam_data[spam_data['target']==0]
    # find all digit in the text
    digit_not_spam = not_spam_dt["text"].str.findall("[0-9]")
    digit_spam = spam_dt["text"].str.findall("[0-9]")
    # get average number
    avg_num_digit_not_spam = np.mean([len(x) for x in digit_not_spam.values])
    avg_num_digit_spam = np.mean([len(x) for x in digit_spam.values])
    
    return (avg_num_digit_not_spam, avg_num_digit_spam)

In [31]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [52]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3))
    X_train_vectorized = vect.fit_transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    # adding features for X_train
    doc_len = X_train.str.len()
    num_digit = [len(x) for x in X_train.str.findall("[0-9]").values]
    X_train_vect_with_len = add_feature(X_train_vectorized, doc_len)
    X_train_final = add_feature(X_train_vect_with_len, num_digit)
    # adding features for X_test
    doc_len_test = X_test.str.len()
    num_digit_test = [len(x) for x in X_test.str.findall("[0-9]").values]
    X_test_vect_with_len = add_feature(X_test_vectorized, doc_len_test)
    X_test_final = add_feature(X_test_vect_with_len, num_digit_test)


    clf = LogisticRegression(C=100)
    clf.fit(X_train_final, y_train)
    predictions = clf.predict(X_test_final)
    
    
    return roc_auc_score(y_test, predictions)

In [53]:
answer_nine()

0.9653328353394565

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [54]:
def answer_ten():

    spam_dt = spam_data[spam_data['target']==1]
    not_spam_dt = spam_data[spam_data['target']==0]
    non_char_not_spam = [len(x) for x in not_spam_dt["text"].str.findall("\W")]
    non_char_spam = [len(x) for x in spam_dt["text"].str.findall("\W")]
    
    return (np.mean(non_char_not_spam), np.mean(non_char_spam))

In [55]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [87]:
def answer_eleven():
    vect = CountVectorizer(min_df=5, analyzer="char_wb", ngram_range=(2,5))
    X_train_vectorized = vect.fit_transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    # adding features for X_train
    doc_len = X_train.str.len()
    num_digit = [len(x) for x in X_train.str.findall("[0-9]").values]
    non_word = [len(x) for x in X_train.str.findall("\W")]
    X_train_vect_with_len = add_feature(X_train_vectorized, doc_len)
    X_train_len_digit = add_feature(X_train_vect_with_len, num_digit)
    X_train_final = add_feature(X_train_len_digit, non_word)

    # adding features for X_test
    doc_len_test = X_test.str.len()
    num_digit_test = [len(x) for x in X_test.str.findall("[0-9]").values]
    non_word_test = [len(x) for x in X_test.str.findall("\W")]
    X_test_vect_with_len = add_feature(X_test_vectorized, doc_len_test)
    X_test_len_digit = add_feature(X_test_vect_with_len, num_digit_test)
    X_test_final = add_feature(X_test_len_digit, non_word_test)

    clf = LogisticRegression(C=100)
    clf.fit(X_train_final, y_train)
    predictions = clf.predict(X_test_final)
    auc = roc_auc_score(y_test, predictions)
    
    # coefficients
    coefs = clf.coef_[0]
    sorted_index = coefs.argsort()
    feature_names = np.array(vect.get_feature_names() +
                             ['length_of_doc', 'digit_count', 'non_word_char_count'])
    smallest_coef = coefs[sorted_index[:10]].tolist()
    largest_coef = coefs[sorted_index[-11:-1][::-1]].tolist()
    
    
    return (auc, smallest_coef, largest_coef)

In [88]:
answer_eleven()

(0.9788593110707434,
 [-0.8697858843712828,
  -0.8608547785182948,
  -0.6769230110354952,
  -0.6670217717778228,
  -0.6148739768660342,
  -0.5795871210576163,
  -0.5350672386161355,
  -0.5057401022100805,
  -0.49851416881046356,
  -0.49093574270688567],
 [0.5977593003266424,
  0.5415001012483578,
  0.5387710472470398,
  0.5214700395309074,
  0.5203564270175837,
  0.517863377170482,
  0.5160830332501773,
  0.5086677067950225,
  0.5026548831862557,
  0.47872837132677976])